In [349]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [350]:
dataset_prime = pd.read_csv('/content/drive/MyDrive/Deep Learning/HW1/agaricus-lepiota.data')
print(dataset_prime.shape)
dataset_prime.head()

(8123, 23)


,p,x,s,n,t,p.1,f,c,n.1,k,e,e.1,s.1,s.2,w,w.1,p.2,w.2,o,p.3,k.1,s.3,u
0,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
1,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
2,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
3,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g
4,e,x,y,y,t,a,f,c,b,n,e,c,s,s,w,w,p,w,o,p,k,n,g


In [351]:
dataset = dataset_prime.copy()


In [352]:
print(np.unique(dataset['e.1']))
print(np.unique(dataset['p.2']))

['?' 'b' 'c' 'e' 'r']
['p']


In [353]:
# dataset = dataset[dataset['e.1'] != '?']
dataset = dataset.drop(columns=['e.1'])
print(dataset.shape)

(8123, 22)


In [354]:
for col in dataset:
  if len(np.unique(dataset[col])) == 1:
    dataset = dataset.drop(columns=[col])

In [355]:
dataset.shape

(8123, 21)

In [356]:
for col in dataset:

  # create dictionary of ordinal to integer mapping
  dict_col = {}
  i = 0
  for el in np.unique(dataset[col]):
    dict_col[el] = i 
    i+=1
  # apply using map
  dataset[col] = dataset[col].map(dict_col)
  # if col != 'p':
  #   dataset[col] = dataset[col]/ 2.6
    
  #   # dataset[col] = (dataset[col] - dataset[col].min(axis=0)) / (dataset[col].max(axis=0) - dataset[col].min(axis=0))

dataset.head()

,p,x,s,n,t,p.1,f,c,n.1,k,e,s.1,s.2,w,w.1,w.2,o,p.3,k.1,s.3,u
0,0,5,2,9,1,0,1,0,0,4,0,2,2,7,7,2,1,4,3,2,1
1,0,0,2,8,1,3,1,0,0,5,0,2,2,7,7,2,1,4,3,2,3
2,1,5,3,8,1,6,1,0,1,5,0,2,2,7,7,2,1,4,2,3,5
3,0,5,2,3,0,5,1,1,0,4,1,2,2,7,7,2,1,0,3,0,1
4,0,5,3,9,1,0,1,0,0,5,0,2,2,7,7,2,1,4,2,2,1


In [357]:

msk = np.random.rand(len(dataset)) < 0.7

train = dataset[msk]

test = dataset[~msk]

In [358]:
test.shape

(2394, 21)

In [359]:
y_train = train['p']
X_train = train.drop(columns=['p'])
y_test = test['p']
X_test = test.drop(columns=['p'])

In [360]:
def accuracy(y_tst,y_prd):
  num_of_corr_lbl = 0

  for yt, yp in zip(y_tst, y_prd):
    if yt == yp:
      num_of_corr_lbl += 1
  acc = num_of_corr_lbl / len(y_tst)
  return acc

In [361]:
def sigmoid(X):
  sig = 1/(1+np.exp(-X))
  return sig


In [362]:
def weight_initialize(number_of_features):
  w = np.random.randn(1,number_of_features)
  b = np.random.randn()
  return w,b

In [363]:
def optimize(X, y, w, b):
  m = X.shape[0]
  y_hat = sigmoid(np.dot(w,X.T)+b)
  
  cost = (-1/m)*(np.sum((y.T*np.log(y_hat)) + ((1-y.T)*(np.log(1-y_hat)))))

  dw = (1/m)*(np.dot(X.T, (y_hat-y.T).T))
  db = (1/m)*(np.sum(y_hat-y.T))
  
  grads = {"dw": dw, "db": db}
  return grads, cost

In [364]:
def logistic_regression(X, y, w, b, learning_rate, number_of_iterations):
  costs = []
  for i in range(number_of_iterations):
    grads, cost = optimize(X, y, w, b)

    dw = grads["dw"]
    db = grads["db"]
    
    w = w - (learning_rate * (dw.T))
    b = b - (learning_rate * db)

    if (i % 50 == 0):
      costs.append(cost)

  params = {"w": w, "b": b}
  gradient = {"dw": dw, "db": db}
  return params, gradient, costs

In [365]:
def predict(X_tst, params):
  w = params["w"]
  b = params["b"]
  m = X_tst.shape[0]
  sig_pred = sigmoid(np.dot(w,X_tst.T)+b)
  y_pred = np.zeros((1,m))
  for i in range(sig_pred.shape[1]):
    if sig_pred[0][i] > 0.5:
      y_pred[0][i] = 1
  return y_pred

In [366]:
number_of_features = X_train.shape[1]
w, b = weight_initialize(number_of_features)

params, gradient, costs = logistic_regression(X_train, np.array(y_train), w, b, learning_rate=0.1,number_of_iterations=3000)

y_trn_pred = predict(X_train, params)
print('Training Accuracy',accuracy(y_trn_pred.T, y_train))
y_tst_pred = predict(X_test, params)
print('Test Accuracy',accuracy(y_tst_pred.T, y_test))

Training Accuracy 0.9439692791063012
Test Accuracy 0.9411027568922306
